<a href="https://colab.research.google.com/github/saishshinde15/CrewAI/blob/main/CrewAi_With_Gradio_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-community crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

llm = ChatCohere()

··········


In [2]:
from crewai import Agent, Task, Crew

In [24]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on a given topic {input}.",
    backstory=(
        "You're working on planning a blog article about the specified topic. "
        "You collect information that helps the audience learn something new "
        "and make informed decisions. "
        "Your work serves as the foundation for the Content Writer to craft "
        "an article on this topic."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [25]:
writer = Agent(
    role="Content Writer",
    goal="Write an insightful and factually accurate opinion piece on a given topic {input}.",
    backstory=(
        "You're working on writing a new opinion piece about the specified topic {input}. "
        "Your writing is based on the work of the Content Planner, who provides an outline "
        "and relevant context. You follow the main objectives and direction of the outline "
        "provided by the Content Planner. "
        "You offer objective and impartial insights, supported by information from the Content Planner. "
        "In your opinion piece, you clearly differentiate between subjective opinions and objective statements."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)


In [26]:
editor = Agent(
    role="Editor",
    goal="Edit a blog post to align with the organization's writing style.",
    backstory=(
        "You are an editor who reviews blog posts submitted by the Content Writer. "
        "Your goal is to ensure the blog post adheres to journalistic best practices, "
        "provides balanced viewpoints when presenting opinions or assertions, "
        "and avoids major controversial topics or opinions when possible."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)


In [27]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on the topic {input}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline, including an introduction, key points, and a call to action.\n"
        "4. Incorporate SEO keywords and relevant data or sources."
    ),
    expected_output=(
        "A comprehensive content plan document with an outline, audience analysis, "
        "SEO keywords, and supporting resources."
    ),
    agent=planner,
)

In [28]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on the given topic {input}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Ensure sections and subtitles are engaging and appropriately named.\n"
        "4. Structure the post with an engaging introduction, an insightful body, and a summarizing conclusion.\n"
        "5. Proofread for grammatical accuracy and ensure alignment with the brand's voice."
    ),
    expected_output=(
        "A well-written blog post in markdown format, ready for publication. "
        "Each section should consist of 2-3 paragraphs."
    ),
    agent=writer,
)


In [29]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [30]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [32]:
input={"input":"Artificial Intelligence"}
result=crew.kickoff(inputs=input)

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on the topic Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline, including an introduction, key points, and a call to action.
4. Incorporate SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
# Content Plan: Unlocking the Potential of Artificial Intelligence

## Introduction:
- Welcome readers to an exploration of the ever-evolving world of Artificial Intelligence (AI).
- Provide a brief overview of AI's impact on various industries and its potential to revolutionize the future.

## Latest Trends in Artificial Intelligence:
1. **Generative AI:**
   - Discuss the rise of Generative AI models like ChatGPT, Midjourney, and Stable Diffusion.
   - Highlight their applications in content creation, art generation, and personalized recommendations.
   - Key Players: Open

In [33]:
print(result)

# Unlocking the Potential of Artificial Intelligence: A Journey into the Future

## Welcome to the AI Revolution

The field of Artificial Intelligence (AI) is an exciting and dynamic realm, constantly evolving and influencing our daily lives. From the moment we wake up to advanced virtual assistants to our daily commute in smart vehicles, AI is an integral part of the modern experience. In this blog post, we delve into the fascinating world of AI, exploring its latest trends, the pioneers driving its development, and the profound impact it has on various industries.

The journey ahead promises to be an eye-opening exploration, revealing how AI is not just a futuristic concept but a powerful force shaping our present and future. Get ready to discover the incredible potential of AI and its ability to revolutionize the way we live and work.

## The AI Landscape: Unlocking Infinite Possibilities

### Generative AI: Igniting Creative Sparks
Generative AI has taken the world by storm, captiv

In [50]:
pip install gradio

In [49]:


import gradio as gr

def generate_blog_post(topic):
    try:
        input_data = {"input": topic}
        result = crew.kickoff(inputs=input_data)
        return result
    except Exception as e:
        return f"An error occurred: {e}"

iface = gr.Interface(
    fn=generate_blog_post,
    inputs=gr.Textbox(label="Enter your topic", lines=2, placeholder="e.g., Artificial Intelligence"),
    outputs="text",
    title="Blog Post Generator",
    description="Generate a blog post based on your input topic."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03e90c9d0025464c39.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
